# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [1]:
import pandas as pd
from pyspark.sql import SparkSession

# Create Spark session
# spark = SparkSession.builder \
#     .master("local[*]") \
#     .appName("MyApp") \
#     .getOrCreate()

spark = (
    SparkSession.builder
    .master("local[*]")
    .appName("MyApp")
    .config("spark.driver.host", "127.0.0.1")
    .config("spark.driver.bindAddress", "127.0.0.1")
    .getOrCreate()
)




25/07/18 13:18:22 WARN Utils: Your hostname, Chloes-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.32.73 instead (on interface en0)
25/07/18 13:18:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/18 13:18:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/07/18 13:18:26 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+------+-------+------------+--------------------+------+-----------+------------+-------------+------------+-------------+-------------+---------------------+-------------------+---------------+--------------------+-------+-----------------------+--------------------+--------------------+---------------------+--------------------+----------+--------------+-------------+----------+-----------+
|Status| Number|Release Date|         Released to|Pieces|Weight (lb)|Volume (ft³)|Creation Date|Income (CAD)|Expense (CAD)|Balance (CAD)|Exclude From Tracking|Volume weight (Vlb)|       Employee|               Notes|Carrier|Commodities Description|   Issued By Address|  Release to Address|Purchase Order Number|      Billing Client|Is Printed|Purchase Order|Delivery date|Lot Number| Container#|
+------+-------+------------+--------------------+------+-----------+------------+-------------+------------+-------------+-------------+---------------------+-------------------+---------------+-----------

In [7]:
df=pd.read_csv('output/commodity_item/commodity.csv')

# Convert pandas → Spark
spark_df= spark.createDataFrame(df)
# Show Spark DataFrame
spark_df.show(truncate=False)


+----+-------+-------------------------+------+--------------+---------------------------+--------------+------------+--------------------------------------------------------+--------------+--------------------------+-----------+----------------------+-------------+---------+---------+--------+---------+----------+---------------+----------------+
|job |status |purchase_order_container#|pieces|contained_item|purchase_order_harvest_date|departure_date|arrival_date|description                                             |model         |item_definition_categories|part_number|purchase_order_pallets|unitary_value|length_in|height_in|width_in|weight_lb|volume_ft³|piece_weight_lb|piece_volume_ft³|
+----+-------+-------------------------+------+--------------+---------------------------+--------------+------------+--------------------------------------------------------+--------------+--------------------------+-----------+----------------------+-------------+---------+---------+--------+-----

In [5]:
spark_df.printSchema()

root
 |-- job: long (nullable = true)
 |-- status: string (nullable = true)
 |-- purchase_order_container#: string (nullable = true)
 |-- pieces: long (nullable = true)
 |-- contained_item: string (nullable = true)
 |-- purchase_order_harvest_date: string (nullable = true)
 |-- departure_date: string (nullable = true)
 |-- arrival_date: string (nullable = true)
 |-- description: string (nullable = true)
 |-- model: string (nullable = true)
 |-- item_definition_categories: string (nullable = true)
 |-- part_number: string (nullable = true)
 |-- purchase_order_pallets: string (nullable = true)
 |-- unitary_value: double (nullable = true)
 |-- length_in: double (nullable = true)
 |-- height_in: double (nullable = true)
 |-- width_in: double (nullable = true)
 |-- weight_lb: string (nullable = true)
 |-- volume_ft³: string (nullable = true)
 |-- piece_weight_lb: double (nullable = true)
 |-- piece_volume_ft³: double (nullable = true)



In [8]:
from pyspark.sql import functions as F

spark_df.createOrReplaceTempView("commodity")


In [81]:
result = spark.sql(''' select job as container, pieces, description,item_definition_categories,part_number,purchase_order_pallets,unitary_value from commodity  order by job asc
''')
result.show(truncate=False)
# status like "On Hand"  AND
# where job BETWEEN 3400 AND 4588 order by job asc
#where status like "On Hand"

+---------+------+------------------------------------------------------------------------------------------------+--------------------------+-----------+----------------------+-------------+
|container|pieces|description                                                                                     |item_definition_categories|part_number|purchase_order_pallets|unitary_value|
+---------+------+------------------------------------------------------------------------------------------------+--------------------------+-----------+----------------------+-------------+
|1953     |144   |Banane Cavendish Equitable Certifiée par Fairtrade (Conventional & Fairtrade)                   |Conventional              |004MXCOL   |americano             |17.49        |
|1953     |480   |Banane Cavendish Bio & Equitable Certifiée par Ecocert Canada et Fairtrade (Organic & Fairtrade)|Organic                   |001MXCOL   |americano             |21.02        |
|1953     |336   |Banane Cavendish Bio &

In [64]:
dfs=pd.read_csv('sales_order_list.csv')

# Convert pandas → Spark
spark_dfs= spark.createDataFrame(dfs)
spark_dfs = spark_dfs.withColumnRenamed("job", "container")
# Show Spark DataFrame
spark_dfs.show(truncate=False)


+-------+--------+-------+----------+---------------------------------------------+---------+--------+------------+-------------+------------+----------------------+--------------------+---------------+-----------+------------+-------------+-------------+-----+--------------------+---------------------------------------------------------------+---------------------------------------------------------------+---------------------------------------------------------------------------------+-------------+------------+---------------------+----+---------------------+-------------------+------------------+---------------------------------------------+--------------------+--------------+--------------+----------+-----------+
|Status |Invoiced|Number |Date      |Buyer                                        |container|Quantity|Amount      |Shipping Date|Amount (CAD)|Unbilled Balance (CAD)|Unpaid Balance (CAD)|Employee       |Weight (lb)|Volume (ft³)|Approved Date|Payment Terms|Notes|Currency   

In [66]:
from pyspark.sql.functions import col, lpad

# Convert to integer, then pad left with 0s to ensure 4 digits
spark_dfs = spark_dfs.withColumn(
    "container",
    lpad(col("container").cast("int").cast("string"), 4, '0')
)

spark_dfs.show(truncate=False)
spark_dfs.createOrReplaceTempView("sales")

+-------+--------+-------+----------+---------------------------------------------+---------+--------+------------+-------------+------------+----------------------+--------------------+---------------+-----------+------------+-------------+-------------+-----+--------------------+---------------------------------------------------------------+---------------------------------------------------------------+---------------------------------------------------------------------------------+-------------+------------+---------------------+----+---------------------+-------------------+------------------+---------------------------------------------+--------------------+--------------+--------------+----------+-----------+
|Status |Invoiced|Number |Date      |Buyer                                        |container|Quantity|Amount      |Shipping Date|Amount (CAD)|Unbilled Balance (CAD)|Unpaid Balance (CAD)|Employee       |Weight (lb)|Volume (ft³)|Approved Date|Payment Terms|Notes|Currency   

In [78]:
res= spark.sql(


    '''
   select  distinct job as container from commodity where job in
    (select distinct container from sales
    where container not like '0000')
    order by container asc

    '''
)
res.show()

+---------+
|container|
+---------+
|     1953|
|     2081|
|     2106|
|     2131|
|     2157|
|     2159|
|     2199|
|     2203|
|     2204|
|     2205|
|     2206|
|     2207|
|     2208|
|     2209|
|     2210|
|     2211|
|     2212|
|     2244|
|     2296|
|     2605|
+---------+
only showing top 20 rows



In [80]:
res= spark.sql(


    '''
    select * from sales
    where container = '1953'


    '''
)
res.show()

+-------+--------+------+----------+--------------------+---------+--------+------------+-------------+------------+----------------------+--------------------+--------------+-----------+------------+-------------+-------------+-----+--------------------+--------------------+--------------------+--------------------+-------------+------------+---------------------+----+---------------------+-------------------+------------------+--------------------+--------------------+--------------+--------------+----------+----------+
| Status|Invoiced|Number|      Date|               Buyer|container|Quantity|      Amount|Shipping Date|Amount (CAD)|Unbilled Balance (CAD)|Unpaid Balance (CAD)|      Employee|Weight (lb)|Volume (ft³)|Approved Date|Payment Terms|Notes|            Currency|     Billing Address|    Shipping Address|      Seller Address|Exchange Rate|Is Cancelled|Reverse Exchange Rate|Paid|Amt. Before Tax (CAD)|Other Charges (CAD)|Items Amount (CAD)|      Billing Client|Total Discount (CA

In [ ]:
res= spark.sql(


    '''
    select * from sales
    where container not like '0000'
    order by container asc

    '''
)
res.show()

In [59]:
dfp=pd.read_csv('output/puchase_fact/Purchase_fact.csv')

# Convert pandas → Spark
spark_dfp= spark.createDataFrame(dfp)
spark_dfp = spark_dfp.withColumnRenamed("job", "container")
# Show Spark DataFrame
spark_dfp.show(truncate=False)
spark_dfp.createOrReplaceTempView("purchase")

+---------+---------+------------------------------------------+------------+----------------+---------------+-----------+--------+-----------------+-----------+----------+--------+--------------------+------------------+------------+------------------+-------------+--------------+------------+---------+-----------------+---------+----------+----------+-------------+------------+-------+----+
|status   |container|supplier                                  |carrier     |port_of_loading |port_of_arrival|container# |quantity|received_quantity|job_expense|job_income|invoiced|unbilled_balance_cad|unpaid_balance_cad|amount      |amt_before_tax_cad|exchange_rate|departure_date|arrival_date|incoterms|customs_clearance|weight_lb|volume_ft³|employee  |shipping_date|job_due_date|date   |paid|
+---------+---------+------------------------------------------+------------+----------------+---------------+-----------+--------+-----------------+-----------+----------+--------+--------------------+------

In [60]:
result_p = spark.sql(
    '''
    SELECT * from
   purchase
   order by container asc

    '''
)

result_p.show()


+---------+---------+--------------------+------------+----------------+---------------+-----------+--------+-----------------+-----------+----------+--------+--------------------+------------------+------------+------------------+-------------+--------------+------------+---------+-----------------+---------+----------+----------+-------------+------------+-------+----+
|   status|container|            supplier|     carrier| port_of_loading|port_of_arrival| container#|quantity|received_quantity|job_expense|job_income|invoiced|unbilled_balance_cad|unpaid_balance_cad|      amount|amt_before_tax_cad|exchange_rate|departure_date|arrival_date|incoterms|customs_clearance|weight_lb|volume_ft³|  employee|shipping_date|job_due_date|   date|paid|
+---------+---------+--------------------+------------+----------------+---------------+-----------+--------+-----------------+-----------+----------+--------+--------------------+------------------+------------+------------------+-------------+-------

In [72]:
df_p = res.toPandas()
df_c = result.toPandas()

In [73]:
common_jobs = set(df_p['container']) & set(df_c['container'])
print("Jobs in both:", common_jobs)


Jobs in both: set()


In [74]:
jobs_p = set(df_p['container'])
jobs_c = set(df_c['container'])

print("Only in df_p:", jobs_p - jobs_c)
print("Only in df_c:", jobs_c - jobs_p)
print("In both:", jobs_p & jobs_c)




Only in df_p: {'4090', '3047', '3125', '3724', '3407', '3877', '4104', '3281', '4082', '3024', '4126', '2176', '2904', '3053', '2531', '2185', '4069', '4025', '3245', '4199', '4448', '3154', '1944', '2112', '2619', '1889', '3521', '4411', '3073', '2920', '3171', '3860', '3510', '4036', '4137', '2832', '3518', '4324', '3885', '3353', '4399', '3303', '2276', '2312', '2289', '3181', '2490', '1933', '4289', '2427', '2208', '3734', '2366', '4142', '2915', '3526', '2170', '2567', '3865', '2637', '2026', '2823', '4077', '3128', '3715', '3280', '2905', '3170', '4493', '2029', '4336', '4389', '3547', '2592', '2608', '4086', '3496', '4269', '2249', '3710', '2812', '3239', '3763', '2839', '3399', '3179', '2520', '3499', '3520', '3610', '2218', '4039', '3836', '3415', '2426', '3921', '1954', '2874', '3206', '3018', '4479', '3731', '2019', '3803', '3030', '3166', '4376', '2232', '4382', '2072', '3640', '2982', '2068', '3691', '3822', '2761', '3775', '4176', '3401', '3718', '2433', '3967', '3334', '

In [29]:
result_p = spark.sql(''' select distinct status from purchase --where status like 'Delivered'
''')
result_p.show()

+----------+
|    status|
+----------+
|In Transit|
| Delivered|
|   On Hand|
|     Empty|
|      Open|
|   Loading|
+----------+



In [3]:
def clean_column_names(sdf):
    """
    Cleans column names of a Spark DataFrame:
    - lowercase
    - replace spaces, colons with underscores
    - remove parentheses and dots
    """
    def clean_name(name):
        return (
            name.lower()
            .replace(" ", "_")
            .replace(":", "_")
            .replace("(", "")
            .replace(")", "")
            .replace(".", "")
        )

    for old_name in sdf.columns:
        sdf = sdf.withColumnRenamed(old_name, clean_name(old_name))

    return sdf


# use the function
spark_df = clean_column_names(spark_df)

spark_df.show(truncate=False)


+------+-------+------------+---------------------------------------------+------+---------+----------+-------------+----------+-----------+-----------+---------------------+-----------------+---------------+-----------------------------------------------------------------------+-------+--------------------------------------------------------+---------------------------------------------------------------------------------+---------------------------------------------------------------+---------------------+---------------------------------------------+----------+--------------+-------------+----------+-----------+
|status|number |release_date|released_to                                  |pieces|weight_lb|volume_ft³|creation_date|income_cad|expense_cad|balance_cad|exclude_from_tracking|volume_weight_vlb|employee       |notes                                                                  |carrier|commodities_description                                 |issued_by_address               

In [17]:
spark_df.columns

['status',
 'number',
 'release_date',
 'released_to',
 'pieces',
 'weight_lb',
 'volume_ft³',
 'creation_date',
 'income_cad',
 'expense_cad',
 'balance_cad',
 'exclude_from_tracking',
 'volume_weight_vlb',
 'employee',
 'notes',
 'carrier',
 'commodities_description',
 'issued_by_address',
 'release_to_address',
 'purchase_order_number',
 'billing_client',
 'is_printed',
 'purchase_order',
 'delivery_date',
 'lot_number',
 'container#']

In [4]:
spark_df.createOrReplaceTempView("cargo_releases")

In [5]:
result = spark.sql(

'''

select job, number, `container#`, pieces, released_to, purchase_order,carrier ,comm, weight_lb,`volume_ft³`,volume_weight_vlb, release_date, creation_date
from (
    select substring(number, 1, 4) AS job, *
    from cargo_releases
    WHERE release_date LIKE '%2023'
       OR release_date LIKE '%2024'
       OR release_date LIKE '%2025'
) as a
where pieces != 0
order by job

'''
)
result.show()


+----+------+----------+------+--------------------+--------------+-------+---------+----------+-----------------+------------+-------------+
| job|number|container#|pieces|         released_to|purchase_order|carrier|weight_lb|volume_ft³|volume_weight_vlb|release_date|creation_date|
+----+------+----------+------+--------------------+--------------+-------+---------+----------+-----------------+------------+-------------+
|2446|  2446|       NaN|     1|Chiovitti Banana ...|        232045|    NaN|42,140.04|  2,359.37|        24,560.22|  04/20/2023|   04/19/2023|
|2452|  2452|       NaN|     1|Chiovitti Banana ...|        232172|    NaN|42,140.04|  2,359.37|        24,560.22|  04/24/2023|   04/21/2023|
|2472| 2472C|       NaN|   624|Chiovitti Banana ...|        232173|    NaN|24,954.92|  1,322.18|        13,763.43|  04/24/2023|   04/21/2023|
|2472| 2472G|       NaN|   336|            GO FRESH|         13504|    NaN|13,437.26|    711.94|         7,411.08|  04/24/2023|   04/21/2023|
|2529|

In [7]:

result.createOrReplaceTempView("cargo_releases_limit")
result_1 = spark.sql('''
select * from cargo_releases_limit
where job like '34%'

''')
result.createOrReplaceTempView("cargo_releases_1")

In [11]:
result_1.show()

+----+-------+-----------+------+--------------------+--------------+--------------------+---------+----------+-----------------+------------+-------------+
| job| number| container#|pieces|         released_to|purchase_order|             carrier|weight_lb|volume_ft³|volume_weight_vlb|release_date|creation_date|
+----+-------+-----------+------+--------------------+--------------+--------------------+---------+----------+-----------------+------------+-------------+
|3400|   3400|ZMOU8841679|     1|          Top Banana|        061924|                 NaN| 3,747.86|  2,359.59|        24,562.48|  06/19/2024|   06/18/2024|
|3401|3401BCH|IKSU5205593|   672|          Sobeys BCH|    4566748040|              Savoie|26,874.53|      0.00|             0.00|  06/17/2024|   06/17/2024|
|3401| 3401QC|IKSU5205593|   288|  Sobeys Quebec City|    4566748045|              Savoie|11,517.65|      0.00|             0.00|  06/17/2024|   06/17/2024|
|3402|   3402|        NaN|     1|Chiovitti Banana ...|    

In [13]:
dfp= pd.read_csv('output/puchase_fact/Purchase_fact.csv')
# Convert pandas → Spark
spark_dfp= spark.createDataFrame(dfp)
spark_dfp.show()


+---------+----+--------------------+------------+----------------+---------------+-----------+--------+-----------------+-----------+----------+--------+--------------------+------------------+------------+------------------+-------------+--------------+------------+---------+-----------------+---------+----------+----------+-------------+------------+-------+----+
|   status| job|            supplier|     carrier| port_of_loading|port_of_arrival| container#|quantity|received_quantity|job_expense|job_income|invoiced|unbilled_balance_cad|unpaid_balance_cad|      amount|amt_before_tax_cad|exchange_rate|departure_date|arrival_date|incoterms|customs_clearance|weight_lb|volume_ft³|  employee|shipping_date|job_due_date|   date|paid|
+---------+----+--------------------+------------+----------------+---------------+-----------+--------+-----------------+-----------+----------+--------+--------------------+------------------+------------+------------------+-------------+--------------+-------

In [14]:
spark_dfp.createOrReplaceTempView("purchase")

In [15]:
rep= spark.sql (
    '''
SELECT p.*, c.carrier as destination_agent, c.`container#`
FROM purchase p
LEFT JOIN (
    SELECT job, carrier, `container#`
    FROM cargo_releases_limit
    WHERE job LIKE '34%' AND carrier != 'NaN'
) c
ON p.job = c.job
ORDER BY p.job

    '''
)

rep.show(truncate=False)

+---------+----+------------------------------------------+------------+----------------+---------------+-----------+--------+-----------------+-----------+----------+--------+--------------------+------------------+------------+------------------+-------------+--------------+------------+---------+-----------------+---------+----------+----------+-------------+------------+-------+----+---------------------+-----------+
|status   |job |supplier                                  |carrier     |port_of_loading |port_of_arrival|container# |quantity|received_quantity|job_expense|job_income|invoiced|unbilled_balance_cad|unpaid_balance_cad|amount      |amt_before_tax_cad|exchange_rate|departure_date|arrival_date|incoterms|customs_clearance|weight_lb|volume_ft³|employee  |shipping_date|job_due_date|date   |paid|destination_agent    |container# |
+---------+----+------------------------------------------+------------+----------------+---------------+-----------+--------+-----------------+------

In [19]:
rep1= spark.sql (
    '''
select* from cargo_releases_limit
    '''
)

rep1.show()
# select carrier as cargo_carrier from cargo_releases_limit
# where carrier != 'NaN'
# group by carrier


+----+------+----------+------+--------------------+--------------+-------+---------+----------+-----------------+------------+-------------+
| job|number|container#|pieces|         released_to|purchase_order|carrier|weight_lb|volume_ft³|volume_weight_vlb|release_date|creation_date|
+----+------+----------+------+--------------------+--------------+-------+---------+----------+-----------------+------------+-------------+
|2446|  2446|       NaN|     1|Chiovitti Banana ...|        232045|    NaN|42,140.04|  2,359.37|        24,560.22|  04/20/2023|   04/19/2023|
|2452|  2452|       NaN|     1|Chiovitti Banana ...|        232172|    NaN|42,140.04|  2,359.37|        24,560.22|  04/24/2023|   04/21/2023|
|2472| 2472C|       NaN|   624|Chiovitti Banana ...|        232173|    NaN|24,954.92|  1,322.18|        13,763.43|  04/24/2023|   04/21/2023|
|2472| 2472G|       NaN|   336|            GO FRESH|         13504|    NaN|13,437.26|    711.94|         7,411.08|  04/24/2023|   04/21/2023|
|2529|

In [10]:

rep1.coalesce(1).write \
    .mode("overwrite") \
    .option("header", True) \
    .csv("output/ground_carrier")

In [35]:
result_with_job = spark.sql("""
SELECT
       substring(number, 1, 4) AS job,
       *
FROM cargo_releases_1


""")

result_with_job.show()

+----+---------+------+------------+--------------------+------+---------+----------+-------------+----------+-----------+-----------+---------------------+-----------------+---------------+-----+--------+-----------------------+--------------------+--------------------+---------------------+--------------------+----------+--------------+-------------+----------+-----------+
| job|   status|number|release_date|         released_to|pieces|weight_lb|volume_ft³|creation_date|income_cad|expense_cad|balance_cad|exclude_from_tracking|volume_weight_vlb|       employee|notes| carrier|commodities_description|   issued_by_address|  release_to_address|purchase_order_number|      billing_client|is_printed|purchase_order|delivery_date|lot_number| container#|
+----+---------+------+------------+--------------------+------+---------+----------+-------------+----------+-----------+-----------+---------------------+-----------------+---------------+-----+--------+-----------------------+---------------

In [46]:
# convert to pandas
df = result_1.toPandas()

In [48]:

print(df['status'].unique())


['Delivered' 'Empty']


In [18]:
dfc= pd. read_csv('Commodity_list.csv')
# Convert pandas → Spark
spark_dfc= spark.createDataFrame(dfc)
spark_dfc.show()

+-------+----------+----------------------+---------------------+----------------------+-------------------------+-----------------------------+---------------------------+-----------------------------+---------------------------+-----------+--------------------------+------+-----------------------+-----------------+----------------------------+----------------------------+------------------------------+------------------+-------------------------+-------------------------------+--------------------------+--------------------------+--------------------------+----------------------+---------------------+----------------------------------+--------------------------------+--------------------------+----------------------+--------------+------------+--------------------+--------------------------------+--------------------+-----------+-----------+----------+-----------+------------+-------------------+-------------+--------------------+-----------------+------------------+--------------+--